In [1]:
import numpy as np

i=0
vectors = []
with open("D:/decoder/vectors_test.txt", "r", encoding='utf-8') as fr:
    for line in fr:
        a = line.replace('\n', '').split('\t')
        vectors.append(list(map(float,a)))
        i+=1
        if i==16:
            break

i=0
text=[]
with open("D:/decoder/texts_test.txt", "r", encoding='utf-8') as fr:
    for line in fr:
        a = line.replace('\n', '')
        text.append(a)
        i+=1
        if i==16:
            break

x_test = np.array(vectors)
vectors=[]

In [2]:
from keras.models import load_model
model = load_model('D:/decoder/models/conv_mse_sen_to_word0.h5')

Using TensorFlow backend.
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [3]:
from bpemb import BPEmb
bpemb_ru = BPEmb(lang="ru", dim=100, vs=3000)

kol= np.zeros(shape=(len(text)))
pieces=[]
for i in range(len(text)):
    s = bpemb_ru.encode_ids(text[i])
    kol[i]+=len(s)
    pieces.append(s)
kol = kol.astype(np.int)
out = np.zeros(shape=(np.sum(kol),100))
buf = 0
for g in range(len(text)):        
    for k in range(0,kol[g]):
        out[buf]+=bpemb_ru.emb.vectors[pieces[g][k]]
        buf+=1

In [6]:
bs=np.zeros(shape=(out.shape[0],100))
buf = 0
for i in range(len(pieces)):
    max_mean_absolute=-1000000000
    min_mean_absolute=1000000000
    max_absolute=-100000000
    min_absolute=100000000
    mean_absolute=0
    
    max_ed=-100000000
    min_ed=100000000
    mean_ed=0
    pred = model.predict(x=x_test[i].reshape(1,512))
    for j in range(kol[i]):
        a=pred[0][j+1]
        #a=model.predict(x=[x_test[i].reshape(1,512),np.array([j+1])])
        bs[buf]=a[1]
        sub = a[1]-out[buf]
        mean_abs=np.sum(sub)/100
        max_abs=np.amax(sub)
        min_abs=np.amin(sub)
        ed = np.linalg.norm(sub)
        print('Токен: '+bpemb_ru.decode_ids([pieces[i][j]])+'\nCредняя абсолютная раздница: ' +str(mean_abs) +'\nМаксимальная абсолютная раздница: '+str(max_abs)+'\nMинммальная абсолютная раздница: '+str(min_abs)+'\nЕвклидово расстояние: '+str(ed)+'\n')
        if mean_abs>max_mean_absolute:
            max_mean_absolute=mean_abs
        if mean_abs<min_mean_absolute:
            min_mean_absolute=mean_abs
        if max_abs>max_absolute:
           max_absolute=max_abs
        if min_abs<min_absolute:
           min_absolute=min_abs
        mean_absolute+=mean_abs
        if ed>max_ed:
            max_ed=ed
        if ed<min_ed:
            min_ed=ed
        mean_ed+=ed
        buf+=1
    mean_absolute/=kol[i]
    mean_ed/=kol[i]
    print('\n\nCредняя абсолютная раздница по предложению: ' +str(mean_absolute)+'\nМаксимальная средняя абсолютная раздница по предложению: '+str(max_mean_absolute)+'\nMинммальная средняя абсолютная раздница по предложению: '+str(min_mean_absolute) +'\nМаксимальная абсолютная раздница по предложению: '+str(max_absolute)+'\nMинммальная абсолютная раздница по предложению: '+str(min_absolute)+'\nСреднее евклидово расстояние по предложению: '+str(mean_ed)+'\nМаксимальное евклидово расстояние в предложении: '+str(max_ed)+'\nМинимальное евклидово расстояние в предложении: '+str(min_ed)+'\n\n\n')

Токен: си
Cредняя абсолютная раздница: -0.05591900595478364
Максимальная абсолютная раздница: 0.534879595041275
Mинммальная абсолютная раздница: -0.5453563928604126
Евклидово расстояние: 2.3931400325929806

Токен: льно
Cредняя абсолютная раздница: -0.09497657570289447
Максимальная абсолютная раздница: 0.6564481258392334
Mинммальная абсолютная раздница: -0.7461268901824951
Евклидово расстояние: 3.1617517419381325

Токен: па
Cредняя абсолютная раздница: -0.15919875865569338
Максимальная абсолютная раздница: 0.4323923885822296
Mинммальная абсолютная раздница: -0.6177915930747986
Евклидово расстояние: 2.6833077986277916

Токен: х
Cредняя абсолютная раздница: -0.20197814041748643
Максимальная абсолютная раздница: 0.40148383378982544
Mинммальная абсолютная раздница: -1.0041781663894653
Евклидово расстояние: 3.294466885041214

Токен: нет
Cредняя абсолютная раздница: -0.13936363263055682
Максимальная абсолютная раздница: 0.6009796261787415
Mинммальная абсолютная раздница: -0.864228367805481
Ев

In [7]:
import math
number = math.factorial(bs.shape[0])/(math.factorial(bs.shape[0]-2)*2)

max =-10000000000000
min=10000000000000
average=0

r = np.zeros(shape=(int(number)))

buf=0
for i in range(bs.shape[0]-1):
    for j in range(i+1,bs.shape[0]):
        b = np.linalg.norm(bs[i] - bs[j])
        if b>max:
            max=b
        if b<min:
            min=b
        average+=b
        r[buf]=b
        buf+=1
        if buf%100000==0:
            print(buf)
average/=number
print('max=',max)
print('min=',min)
print('average=',average)

max= 16.182662546634674
min= 2.086162567138672e-06
average= 3.779452353981503
